In [1]:
#!/usr/bin/env python
import os
import sys
import rospy
import copy
import tf
import cv2
from cv_bridge import CvBridge, CvBridgeError
import numpy as np
import std_msgs.msg
from std_msgs.msg import UInt16, String
import geometry_msgs.msg
from apriltags_ros.msg import AprilTagDetection, AprilTagDetectionArray
from math import *
from numpy.linalg import inv
import scipy.io
from mat4py import loadmat
import actionlib
from robotiq_2f_gripper_msgs.msg import CommandRobotiqGripperFeedback, CommandRobotiqGripperResult, CommandRobotiqGripperAction, CommandRobotiqGripperGoal
from robotiq_2f_gripper_control.robotiq_2f_gripper_driver import Robotiq2FingerGripperDriver as Robotiq
from scoop.msg import stone_pose_hori_learning_used
from scoop.msg import stone_pose_ver_learning_used
from sensor_msgs.msg import Image, CameraInfo
import message_filters

from visualization_msgs.msg import Marker
from visualization_msgs.msg import MarkerArray
from geometry_msgs.msg import Point
import random
import urx
import math3d as m3d
import logging

import matplotlib.pyplot as plt
import torch

#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
rospy.init_node('scoop', anonymous=True) #initialize the node



In [2]:
rospy.logwarn("Client test: Starting sending goals")
rob = urx.Robot("192.168.1.102")
logging.basicConfig(level=logging.WARN)
rob.set_tcp((0, 0, 0, 0, 0, 0))

[WARN] [1607070311.719825]: Client test: Starting sending goals


In [3]:
action_name = rospy.get_param('~action_name', 'command_robotiq_action')
robotiq_client = actionlib.SimpleActionClient(action_name, CommandRobotiqGripperAction)
robotiq_client.wait_for_server()
servo_pub = rospy.Publisher('servo1', UInt16, queue_size=10)

In [4]:
servo2_pub = rospy.Publisher('servo2', UInt16, queue_size=10)

def softtip():
    servo2_pub.publish(60) #bringing soft-pad at the tip
    rospy.sleep(1.8)
    servo2_pub.publish(90)
    
def hardtip():
    servo2_pub.publish(120) #bringing soft-pad at the tip
    rospy.sleep(1.8)
    servo2_pub.publish(90)

In [5]:
def go_to_home():
    
    Hong_joint0 = radians(102.02)
    Hong_joint1 = radians(-83.71)
    Hong_joint2 = radians(-130.34)
    Hong_joint3 = radians(-55.99)
    Hong_joint4 = radians(89.33)
    Hong_joint5 = radians(-77.94)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_safe_place():
    
    Hong_joint0 = radians(101.74)
    Hong_joint1 = radians(-76.84)
    Hong_joint2 = radians(-120.83)
    Hong_joint3 = radians(-72.33)
    Hong_joint4 = radians(89.40)
    Hong_joint5 = radians(-78.12)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)
    
def go_to_end_place():
    
    Hong_joint0 = radians(106.13)
    Hong_joint1 = radians(-60.48)
    Hong_joint2 = radians(-130.69)
    Hong_joint3 = radians(-78.86)
    Hong_joint4 = radians(89.68)
    Hong_joint5 = radians(-73.78)
    
    rob.movej((Hong_joint0,Hong_joint1, Hong_joint2, Hong_joint3, Hong_joint4, Hong_joint5), 0.6, 1.5)

In [6]:
def stone_position_in_the_world(pose_x, pose_y, pose_z):
    if pose_z==0.0:
        return [None, None, None]
    rob.set_tcp((0, 0.0, 0, 0, 0, 0))    
    camPstone = np.array([pose_x, pose_y, pose_z, 1])
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    eeTstone = np.matmul(eeTcam.get_matrix(), camPstone)
    tcpTstone = np.matmul(tcpTee, np.transpose(eeTstone))
    worldTstone = np.matmul(rob.get_pose().get_matrix(), tcpTstone)
    worldTstone = worldTstone.squeeze().tolist()[0][0:-1]
    return [worldTstone[0]+0.01953+0.0026, worldTstone[1]+0.02148+0.0029-0.0012, worldTstone[2]]

In [7]:
def hori_stone_pose_callback(msg):
    global hori_pose_Go_stone_set, hori_Go_stone_number, yaw_pitch_normal_set, hori_gripper_direction_set, hori_gripper_aperture_set
    hori_Go_stone_number = int(msg.hori_Go_stone_number)
    if hori_Go_stone_number == 0:
        hori_pose_Go_stone_set = []
        Go_stone_dir_sorted = []
        yaw_pitch_normal_set = []
        hori_gripper_direction_set = []
        hori_gripper_aperture_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z
        yaw = msg.yaw
        pitch = msg.pitch
        normal = msg.normal
        hori_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        yaw_pitch_normal_set.append([yaw,pitch,normal])
        box = json.loads(msg.box)
        box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
        #print box
        length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
        length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
        if length1 > length2:
            try:
                if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                hori_gripper_direction_set.append(temp_angle)
            except:
                hori_gripper_direction_set.append(0)
            hori_gripper_aperture_set.append(length2+0.004)
        else:
            try:
                if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                hori_gripper_direction_set.append(temp_angle)
            except:
                hori_gripper_direction_set.append(0)
            hori_gripper_aperture_set.append(length1+0.004)        
hori_pose_topic = '/hori_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(hori_pose_topic, stone_pose_hori_learning_used, hori_stone_pose_callback)

In [8]:
import json
def ver_stone_pose_callback(msg):
    global ver_pose_Go_stone_set, ver_Go_stone_number, ver_gripper_direction_set, ver_gripper_aperture_set
    ver_Go_stone_number = int(msg.ver_Go_stone_number)
    if ver_Go_stone_number == 0:
        ver_pose_Go_stone_set = []
        ver_gripper_direction_set = []
        ver_gripper_aperture_set = []
    else:
        pose_x = msg.x
        pose_y = msg.y
        pose_z = msg.z 
        box = json.loads(msg.box)
        ver_pose_Go_stone_set.append(stone_position_in_the_world(pose_x, pose_y, pose_z))
        box = [stone_position_in_the_world(box[i][0], box[i][1], pose_z)[0:2] for i in range(len(box))]
        #print box
        length1 = np.sqrt((box[0][0]-box[1][0])**2+(box[0][1]-box[1][1])**2)
        length2 = np.sqrt((box[2][0]-box[1][0])**2+(box[2][1]-box[1][1])**2)
        if length1 > length2:
            try:
                if abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2)<abs(atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[0][0]-box[1][0])/(box[0][1]-box[1][1]))-pi/2
                ver_gripper_direction_set.append(temp_angle)
            except:
                ver_gripper_direction_set.append(0)
            ver_gripper_aperture_set.append(length2+0.004)
        else:
            try:
                if abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2)<abs(atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2):
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))+pi/2
                else:
                    temp_angle=atan((box[2][0]-box[1][0])/(box[2][1]-box[1][1]))-pi/2
                ver_gripper_direction_set.append(temp_angle)
            except:
                ver_gripper_direction_set.append(0)
            ver_gripper_aperture_set.append(length1+0.004)        
ver_pose_topic = '/ver_stone_pose' #"/usb_cam/image_raw"
rospy.Subscriber(ver_pose_topic, stone_pose_ver_learning_used, ver_stone_pose_callback)

In [9]:
is_send_stone_img = 0
stone_img_count = 0
bridge = CvBridge()

def image_callback(color, a_depth):
    global is_send_stone_img, stone_img_count, cv2_img, cv2_depth_img
    if is_send_stone_img == 1:
        print "dsag"
        img_count_pub = rospy.Publisher('/stone_img_index', String, queue_size=1)
        rospy.sleep(0.5)
        cv2_img = bridge.imgmsg_to_cv2(color, "bgr8")
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/JPEGImages/'+str(stone_img_count)+'.jpeg', cv2_img)
        cv2_depth_img = bridge.imgmsg_to_cv2(a_depth, desired_encoding="passthrough")
        depth_array = np.array(cv2_depth_img, dtype=np.float32)
        cv2.imwrite('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.jpeg', cv2_depth_img)
        np.save('/home/terry/Mask_RCNN/samples/stones/depth/'+str(stone_img_count)+'.npy', depth_array)
        img_count_pub.publish(str(stone_img_count))
        print str(stone_img_count)
        stone_img_count = stone_img_count + 1
        is_send_stone_img = 0

In [10]:
image_color_sub = message_filters.Subscriber('/camera/color/image_raw', Image)
image_aligned_depth_sub = message_filters.Subscriber('/camera/aligned_depth_to_color/image_raw', Image)
ts = message_filters.TimeSynchronizer([image_color_sub, image_aligned_depth_sub], 10)
ts.registerCallback(image_callback)

0

In [11]:
def poke_grasp_collision_check(instance_index):
    global hori_pose_Go_stone_set, yaw_pitch_normal_set
    if instance_index[0]=='v':
        return True
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    normal=yaw_pitch_normal_set[instance_index[1]][2]
    n_normalize = normal / np.linalg.norm(normal)
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.004
    world2tcp.pos.y+=0.001
    world2normal = np.matmul(np.matmul(world2tcp.get_matrix(),tcp2cam), n_normalize).tolist()[0]
    world2finger_normal = [0,0,0]
    world2finger_normal[2] = sin(atan2(world2normal[2], sqrt(world2normal[0]**2+world2normal[1]**2))+radians(27))
    if atan2(world2normal[2], sqrt(world2normal[0]**2+world2normal[1]**2))+radians(27)>pi/2:
        world2finger_normal[0]=-world2normal[0]/sqrt(world2normal[0]**2+world2normal[1]**2)*sqrt(1-world2finger_normal[2]**2)
        world2finger_normal[1]=-world2normal[1]/sqrt(world2normal[0]**2+world2normal[1]**2)*sqrt(1-world2finger_normal[2]**2)
    else:
        world2finger_normal[0]=world2normal[0]/sqrt(world2normal[0]**2+world2normal[1]**2)*sqrt(1-world2finger_normal[2]**2)
        world2finger_normal[1]=world2normal[1]/sqrt(world2normal[0]**2+world2normal[1]**2)*sqrt(1-world2finger_normal[2]**2)
    world2finger_normal=[-world2finger_normal[0], -world2finger_normal[1], -world2finger_normal[2]]
    if Point(hori_pose_Go_stone_set[instance_index[1]][0]+world2finger_normal[0]*0.026, hori_pose_Go_stone_set[instance_index[1]][1]+world2finger_normal[1]*0.026).within(Point([0.05,0.69]).buffer(0.055))==False:
        print 'ccresult', True
        return True
    if hori_pose_Go_stone_set[instance_index[1]][2]+world2finger_normal[2]*0.026<0.03:
        print 'ccresult', True
        return True
    print 'ccresult', False
    return False
    

In [12]:
def poke_grasp_execute(instance_index):    
    global hori_pose_Go_stone_set, yaw_pitch_normal_set
    if instance_index[0]=='v':
        return False
    tcp2fingertip=m3d.Transform()
    tcp2fingertip.pos.x = 0.002
    tcp2fingertip.pos.y = 0  
    tcp2fingertip.pos.z = 0.356
    fingertip2tcp = np.linalg.inv(tcp2fingertip.get_matrix())
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    normal=yaw_pitch_normal_set[instance_index[1]][2]
    n_normalize = normal / np.linalg.norm(normal)
    n_normalize = np.array([n_normalize[0], n_normalize[1], n_normalize[2], 1])
    fingertip2normal = np.matmul(np.matmul(fingertip2tcp, tcp2cam), n_normalize).tolist()[0]
    alpha = atan2(fingertip2normal[1], fingertip2normal[0])
    if str(alpha) == 'nan':
        return False
    beta = -(pi/2+atan2(fingertip2normal[2], sqrt(fingertip2normal[0]**2+fingertip2normal[1]**2)))
    if str(beta) == 'nan':
        return False
    if abs(beta)>(35*pi/180.0):
        return False
    hardtip()
    go_to_safe_place()
    servo_pub.publish(100)
    rob.movel_tool((0,0,0,0,0,alpha), acc=0.5, vel=0.8, wait=True)
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.004
    world2tcp.pos.y+=0.001
    world2fingertip=(world2tcp*tcp2fingertip)
    rob.translate((hori_pose_Go_stone_set[instance_index[1]][0]-world2fingertip.pos.x,hori_pose_Go_stone_set[instance_index[1]][1]-world2fingertip.pos.y,0), acc=0.6, vel=1.5, wait=True)
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.004
    world2tcp.pos.y+=0.001
    world2fingertip=(world2tcp*tcp2fingertip)
    fingertip2stone=np.transpose(np.matmul(np.linalg.inv(world2fingertip.get_matrix()), np.transpose(np.array([hori_pose_Go_stone_set[instance_index[1]]+[1]]))))[0].tolist()[0][0:-1]
    rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
    rospy.sleep(0.5)
    #if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
    rob.movel_tool((0,0,0,0,beta,0), acc=0.5, vel=0.8, wait=True)
    #rob.translate_tool((0.004+0.0015*int(abs(beta)>15)+0.0025*int(abs(beta)>20), 0, 0), acc=0.06, vel=0.08)
    rob.translate_tool((0.003, 0, 0), acc=0.1, vel=0.1)
    psi = radians(27)
    rob.movel_tool((0,0,0,0,psi,0), acc=0.3, vel=0.8, wait=True)
    rob.translate((0, 0, -(fingertip2stone[2]-0.010)), acc=0.1, vel=0.1, wait=True)
    rob.translate_tool((0, 0, 0.026), acc=0.01, vel=0.05)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.2, force=1, block=False)
    rospy.sleep(1)
    rob.translate_tool((0, 0, -0.08), acc=0.3, vel=0.8)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    return True

In [13]:
from shapely.geometry import LineString
from shapely.geometry import Point

def scoop_collision_check(instance_index, ori_index):
    global hori_pose_Go_stone_set, ver_pose_Go_stone_set
    gripper_orientation = ori_index*45
    if instance_index[0]=='h':
        pose_Go_stone_set = hori_pose_Go_stone_set
    elif instance_index[0]=='v':
        pose_Go_stone_set = ver_pose_Go_stone_set
    hard_fingertip_position = [pose_Go_stone_set[instance_index[1]][0]+0.013*sin(ori_index*45*pi/180), pose_Go_stone_set[instance_index[1]][1]-0.013*cos(ori_index*45*pi/180)]
    hard_fingertip_position1 = [hard_fingertip_position[0]+0.01*cos(gripper_orientation*pi/180), hard_fingertip_position[1]+0.01*sin(gripper_orientation*pi/180)]
    hard_fingertip_position2 = [hard_fingertip_position[0]-0.01*cos(gripper_orientation*pi/180), hard_fingertip_position[1]-0.01*sin(gripper_orientation*pi/180)]
    if LineString([hard_fingertip_position1, hard_fingertip_position, hard_fingertip_position2]).within(Point([0.05,0.69]).buffer(0.055))==False:
        return True
    soft_fingertip_position = [pose_Go_stone_set[instance_index[1]][0]+(-0.015)*sin(ori_index*45*pi/180), pose_Go_stone_set[instance_index[1]][1]-(-0.015)*cos(ori_index*45*pi/180)]
    if LineString([[soft_fingertip_position[0]+0.01*cos(gripper_orientation*pi/180), soft_fingertip_position[1]+0.01*sin(gripper_orientation*pi/180)], soft_fingertip_position, [soft_fingertip_position[0]-0.01*cos(gripper_orientation*pi/180), soft_fingertip_position[1]-0.01*sin(gripper_orientation*pi/180)]]).within(Point([0.05,0.69]).buffer(0.055))==False:
        return True
    soft_fingertip_position2 = [pose_Go_stone_set[instance_index[1]][0]+(-0.018)*sin(ori_index*45*pi/180), pose_Go_stone_set[instance_index[1]][1]-(-0.018)*cos(ori_index*45*pi/180)]
    if LineString([[soft_fingertip_position2[0]+0.01*cos(gripper_orientation*pi/180), soft_fingertip_position2[1]+0.01*sin(gripper_orientation*pi/180)], soft_fingertip_position2, [soft_fingertip_position2[0]-0.01*cos(gripper_orientation*pi/180), soft_fingertip_position2[1]-0.01*sin(gripper_orientation*pi/180)]]).within(Point([0.05,0.69]).buffer(0.055))==False:
        return True
    fingertip_extend_position = [pose_Go_stone_set[instance_index[1]][0]+0.15*sin(ori_index*45*pi/180), pose_Go_stone_set[instance_index[1]][1]-0.15*cos(ori_index*45*pi/180)]
    fingertip_extend_position1 = [fingertip_extend_position[0]+0.01*cos(gripper_orientation*pi/180), fingertip_extend_position[1]+0.01*sin(gripper_orientation*pi/180)]
    fingertip_extend_position2 = [fingertip_extend_position[0]-0.01*cos(gripper_orientation*pi/180), fingertip_extend_position[1]-0.01*sin(gripper_orientation*pi/180)]
    intersect_at_bowl = list(Point([0.05,0.69]).buffer(0.065).boundary.intersection(LineString([hard_fingertip_position, fingertip_extend_position])).coords[:][0])
    intersect_at_bowl1 = list(Point([0.05,0.69]).buffer(0.065).boundary.intersection(LineString([hard_fingertip_position1, fingertip_extend_position1])).coords[:][0])
    intersect_at_bowl2 = list(Point([0.05,0.69]).buffer(0.065).boundary.intersection(LineString([hard_fingertip_position2, fingertip_extend_position2])).coords[:][0])
    if Point(hard_fingertip_position).distance(Point(intersect_at_bowl))*sqrt(3)<0.07-(pose_Go_stone_set[instance_index[1]][2]-0.03): # need to be changed
        return True
    if Point(hard_fingertip_position1).distance(Point(intersect_at_bowl1))*sqrt(3)<0.07-(pose_Go_stone_set[instance_index[1]][2]-0.03): # need to be changed
        return True
    if Point(hard_fingertip_position2).distance(Point(intersect_at_bowl2))*sqrt(3)<0.07-(pose_Go_stone_set[instance_index[1]][2]-0.03): # need to be changed
        return True
    return False           

In [14]:
from geometry_msgs.msg import WrenchStamped
from std_msgs.msg import UInt16
import time

def finger_tip_position_wrt_gripper_frame(alpha, l0, l1, l2l, l2r, flength_l, flength_r):
    alpha=alpha*pi/180
    left_fingertip_position=[-l0-l1*sin(alpha)+l2l, -l1*cos(alpha)-flength_l]
    right_fingertip_position=[l0+l1*sin(alpha)-l2r, -l1*cos(alpha)-flength_r]
    return left_fingertip_position, right_fingertip_position

def pos_flength_adjust_left_fixed(current_alpha, next_alpha, theta, l0, l1, l2l, l2r, flength_l):
    theta=theta*pi/180
    current_flength_r = flength_l + (2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/tan(theta)
    current_left_fingertip_pos_g, current_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(current_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    transforming_matrix_w_g = np.array([[sin(theta), -cos(theta), -current_left_fingertip_pos_g[0]*sin(theta)+current_left_fingertip_pos_g[1]*cos(theta)], [cos(theta), sin(theta), -current_left_fingertip_pos_g[0]*cos(theta)-current_left_fingertip_pos_g[1]*sin(theta)], [0, 0, 1]])
    current_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[current_right_fingertip_pos_g[0]], [current_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_left_fingertip_pos_g, next_right_fingertip_pos_g = finger_tip_position_wrt_gripper_frame(next_alpha, l0, l1, l2l, l2r, flength_l, current_flength_r)
    next_left_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_left_fingertip_pos_g[0]], [next_left_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    next_right_fingertip_pos_w = (np.squeeze(np.dot(transforming_matrix_w_g, np.array([[next_right_fingertip_pos_g[0]], [next_right_fingertip_pos_g[1]], [1]])))[0:2]).tolist()
    translation_dir_dis = [-next_left_fingertip_pos_w[0], -next_left_fingertip_pos_w[1]]
    next_right_fingertip_pos_after_trans_w = [next_right_fingertip_pos_w[0]+translation_dir_dis[0], next_right_fingertip_pos_w[1]+translation_dir_dis[1]]
    right_finger_shorten_distance=abs(next_right_fingertip_pos_after_trans_w[1])/sin(theta)
    translation_dir_dis[0] = translation_dir_dis[0]+(2*l0+2*l1*sin(current_alpha*pi/180)-l2l-l2r)/sin(theta)-(next_right_fingertip_pos_after_trans_w[0]-right_finger_shorten_distance*cos(theta))
    return translation_dir_dis, right_finger_shorten_distance

def scoop_execute(instance_index, ori_index):
    global hori_pose_Go_stone_set, ver_pose_Go_stone_set
    gripper_orientation = ori_index*45
    if instance_index[0]=='h':
        hard_fingertip_position = [hori_pose_Go_stone_set[instance_index[1]][0]+0.013*sin(ori_index*45*pi/180), hori_pose_Go_stone_set[instance_index[1]][1]-0.013*cos(ori_index*45*pi/180)]
    elif instance_index[0]=='v':
        hard_fingertip_position = [ver_pose_Go_stone_set[instance_index[1]][0]+0.013*sin(ori_index*45*pi/180), ver_pose_Go_stone_set[instance_index[1]][1]-0.013*cos(ori_index*45*pi/180)]
    softtip()
    go_to_safe_place()
    Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
    servo_pub.publish(0)
    rob.movel_tool((0,0,0,0,0,-gripper_orientation*pi/180),acc=0.5, vel=2)
    tcp2fingertip=m3d.Transform()
    tcp2fingertip.pos.x = -0.021 # need change
    tcp2fingertip.pos.z = 0.342 # need change
    world2tcp = rob.get_pose()
    world2tcp.pos.x+=0.004
    world2tcp.pos.y+=0.001
    world2fingertip=(world2tcp*tcp2fingertip)
    rob.translate((hard_fingertip_position[0]-world2fingertip.pos.x,hard_fingertip_position[1]-world2fingertip.pos.y,0), acc=0.5, vel=2, wait=True)
    rob.set_tcp((-0.021, 0, 0.342, 0, 0, 0))
    rospy.sleep(0.5)
    rob.movel_tool((0,0,0,0,pi/6,0), acc=0.5, vel=2, wait=True)
    rob.set_tcp((0, 0, 0, 0, 0, 0))
    rospy.sleep(0.5)
    if instance_index[0]=='h':
        rob.translate((0, 0, hori_pose_Go_stone_set[instance_index[1]][2]-world2fingertip.pos.z+0.002), acc=0.02, vel=0.05, wait=True)
    elif instance_index[0]=='v':
        rob.translate((0, 0, ver_pose_Go_stone_set[instance_index[1]][2]-world2fingertip.pos.z+0.002), acc=0.02, vel=0.05, wait=True)
    rob.translate((0.005*sin(-gripper_orientation*pi/180), 0.005*cos(-gripper_orientation*pi/180), 0), acc=0.01, vel=0.01, wait=True)
    for current_finger_length in np.arange(0,60,1):  #need change
        servo_pub.publish(current_finger_length)
        rospy.sleep(0.02) #need change
    current_fingerlength_int=60   #need change
    angle_step=0.2
    for angle in np.arange(5,-4,-angle_step):
        translate_dir_dis, shorten_distance = pos_flength_adjust_left_fixed(angle, angle-angle_step, 60, 0.0125, 0.1, 0.006, 0.019, 0.072)
        old_fingerlength_int=current_fingerlength_int
        current_fingerlength_int=round(max(current_fingerlength_int-shorten_distance*(160.0/0.025),0))
        servo_pub.publish(int(current_fingerlength_int))
        if abs(current_fingerlength_int-old_fingerlength_int)!=0:
            Robotiq.goto(robotiq_client, pos=(angle-1)*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
            rob.translate_tool((sin(60*pi/180)*translate_dir_dis[0]+cos(60*pi/180)*translate_dir_dis[1], 0, cos(60*pi/180)*translate_dir_dis[0]-sin(60*pi/180)*translate_dir_dis[1]), acc=0.1, vel=0.4, wait=True)
    Robotiq.goto(robotiq_client, pos=-4*0.14/48.0+1.12/48.0, speed=0.2, force=10, block=False)
    rospy.sleep(0.5)
    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)

In [15]:
def find_alpha_from_distance(distance, l0, l1, l2l, l2r):
    alpha = asin((distance+l2l+l2r-2*l0)/2/l1)*180/pi
    return alpha

def direct_pick_execute(instance_index):
    global hori_pose_Go_stone_set, ver_pose_Go_stone_set, hori_gripper_direction_set, hori_gripper_aperture_set, ver_gripper_direction_set, ver_gripper_aperture_set
    if instance_index[0]=='h':
        pose_Go_stone_set = hori_pose_Go_stone_set
        gripper_direction_set = hori_gripper_direction_set
        gripper_aperture_set = hori_gripper_aperture_set
    elif instance_index[0]=='v':
        pose_Go_stone_set = ver_pose_Go_stone_set
        gripper_direction_set = ver_gripper_direction_set
        gripper_aperture_set = ver_gripper_aperture_set
    go_to_safe_place()
    hardtip()
    Robotiq.goto(robotiq_client, pos=0.75*gripper_aperture_set[instance_index[1]]+0.0095, speed=0.1, force=10, block=True)
    servo_pub.publish(10)    
    pos_x = rob.get_pose().pos.x+0.007
    pos_y = rob.get_pose().pos.y-0.001
    rospy.sleep(0.5)
    rob.translate((pose_Go_stone_set[instance_index[1]][0]-pos_x,pose_Go_stone_set[instance_index[1]][1]-pos_y,0), acc=0.8, vel=2, wait=True)
    rob.movel_tool((0,0,0,0,0,gripper_direction_set[instance_index[1]]),acc=0.8, vel=2)
    rob.translate_tool((0.005,0,0),acc=0.1, vel=0.2)
    #rob.translate((0,0,-0.025), acc=0.1, vel=0.4, wait=True)
    rob.translate((0,0,max((pose_Go_stone_set[instance_index[1]][2]-(rob.get_pose().pos.z-0.32210))*0.7,-0.065)), acc=0.1, vel=0.1, wait=True)
    rob.translate((0,0,(pose_Go_stone_set[instance_index[1]][2]-(rob.get_pose().pos.z-0.32210))*0.3-0.003), acc=0.001, vel=0.01, wait=True)
    Robotiq.goto(robotiq_client, pos=-7*0.14/48.0+1.12/48.0, speed=0.1, force=0.3, block=False)
    rospy.sleep(1)
    rob.translate((0, 0, 0.08), acc=0.3, vel=0.8)

In [16]:
from shapely.affinity import rotate
from shapely.affinity import translate
from shapely.geometry import Point
from shapely.geometry import LinearRing

def direct_pick_collision_check(instance_index):
    global hori_pose_Go_stone_set, ver_pose_Go_stone_set, hori_gripper_direction_set, hori_gripper_aperture_set, ver_gripper_direction_set, ver_gripper_aperture_set
    if instance_index[0]=='h':
        pose_Go_stone_set = hori_pose_Go_stone_set
        gripper_direction_set = hori_gripper_direction_set
        gripper_aperture_set = hori_gripper_aperture_set
    elif instance_index[0]=='v':
        pose_Go_stone_set = ver_pose_Go_stone_set
        gripper_direction_set = ver_gripper_direction_set
        gripper_aperture_set = ver_gripper_aperture_set
    finger_profile = [[0.01, gripper_aperture_set[instance_index[1]]/2+0.019], [-0.01, gripper_aperture_set[instance_index[1]]/2+0.019], [-0.01, -gripper_aperture_set[instance_index[1]]/2-0.007], [0.01, -gripper_aperture_set[instance_index[1]]/2-0.007]]
    finger_profile = rotate(LinearRing(finger_profile), -gripper_direction_set[instance_index[1]], origin=Point([0,0]), use_radians=True)
    finger_profile = translate(LinearRing(finger_profile), xoff=pose_Go_stone_set[instance_index[1]][0], yoff=pose_Go_stone_set[instance_index[1]][1], zoff=0.0)
    if finger_profile.within(Point([0.05,0.69]).buffer(0.055))==True:
        return False
    else:
        return True

In [17]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.utils.model_zoo as model_zoo

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):

    def __init__(self, block, layers, input_image_dim, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(input_image_dim, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

def resnet18(pretrained=False, input_image_dim=3, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], input_image_dim, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

def resnet34(pretrained=False, input_image_dim=3, **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], input_image_dim, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model

def resnet50(pretrained=False, input_image_dim=3, **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], input_image_dim, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

def resnet101(pretrained=False, input_image_dim=3, **kwargs):
    """Constructs a ResNet-101 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], input_image_dim, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model

def resnet152(pretrained=False, input_image_dim=3, **kwargs):
    """Constructs a ResNet-152 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], input_image_dim, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet18_image = resnet18(input_image_dim=3)
        self.resnet18_depth = resnet18(input_image_dim=1)
        self.resnet18_mask = resnet18(input_image_dim=1)
        self.conv1 = nn.Sequential(nn.Conv1d(3, 16, 5, 1, 2), nn.ReLU(), nn.MaxPool1d(4))  
        self.conv2 = nn.Sequential(nn.Conv1d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool1d(4))
        self.conv3 = nn.Sequential(nn.Conv1d(32, 64, 5, 1, 2), nn.ReLU(), nn.MaxPool1d(4))
        self.fc = nn.Sequential(nn.Linear(960, 20), nn.ReLU(), nn.Linear(20, 1))

    def forward(self, input_image, input_depth, input_mask):
        input_image_resnet = torch.unsqueeze(self.resnet18_image(input_image),1)
        input_depth_resnet = torch.unsqueeze(self.resnet18_depth(input_depth),1)
        input_mask_resnet = torch.unsqueeze(self.resnet18_mask(input_mask),1)
        x = torch.cat((input_image_resnet, input_depth_resnet, input_mask_resnet), dim=1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = x.view(x.size(0), -1)
        output = self.fc(x)
        return output


#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
#device = 'cuda:0'
net_dict = dict()
for action_name in ['ad_hoc_pick', 'dig_grasp', 'scoop_dir0', 'scoop_dir1', 'scoop_dir2', 'scoop_dir3', 'scoop_dir4', 'scoop_dir5', 'scoop_dir6', 'scoop_dir7']:
    net_dict[action_name] = Net()
    state_dict = torch.load('Neural_networks/ResNet_'+action_name+'.pkl', map_location=lambda storage, loc: storage)
    state_dict = {".".join(k.split(".")[1:]): v for k,v in state_dict.items()}
    net_dict[action_name].load_state_dict(state_dict)
    

#     net_dict[action_name].load_state_dict(torch.load('Neural_networks/ResNet_'+action_name+'.pkl', map_location=lambda storage, loc: storage))
#     print("**************start*****************")
#     model = net_dict[action_name]
#     for name, layer in model.named_parameters():
#         print(name)
#     print(torch.load('Neural_networks/ResNet_'+action_name+'.pkl', map_location=lambda storage, loc: storage).keys())
#     print("**************end*******************")
    #net_dict[action_name].module

In [26]:
from torch.autograd import Variable

#device = 'cuda:0'
stone_img_count = 0
picking_count = 0
dir_name = time.asctime(time.localtime(time.time())).replace(':', '_').replace(' ','_')
os.makedirs('collecting_data/'+dir_name)
os.makedirs('collecting_data/'+dir_name+'/image')
os.makedirs('collecting_data/'+dir_name+'/depth')
os.makedirs('collecting_data/'+dir_name+'/mask')
while True:
    whether_successful = False
    whether_executed = False
    while True:
        if_continue = raw_input("continue?")
        if if_continue != '':
            break
    go_to_safe_place()
    servo_pub.publish(10)
    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
    rospy.sleep(1)
    go_to_home()
    rospy.sleep(1)
    hori_Go_stone_number = None
    ver_Go_stone_number = None
    hori_pose_Go_stone_set = []
    yaw_pitch_normal_set = []
    hori_gripper_direction_set = []
    hori_gripper_aperture_set = []
    ver_pose_Go_stone_set = []
    ver_gripper_direction_set = []
    ver_gripper_aperture_set = []
    is_send_stone_img = 1
    print 'started', stone_img_count
    while True:
        if hori_Go_stone_number != None and hori_Go_stone_number == len(hori_pose_Go_stone_set) and ver_Go_stone_number != None and ver_Go_stone_number == len(ver_pose_Go_stone_set):
            break
        rospy.sleep(0.1)   
    print hori_Go_stone_number, hori_pose_Go_stone_set
    print ver_Go_stone_number, ver_pose_Go_stone_set
    instance_list = []
    if ver_pose_Go_stone_set == []:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            instance_list.append(('h', index_hori))
    elif hori_pose_Go_stone_set == []:
        for index_ver in range(len(ver_pose_Go_stone_set)):
            instance_list.append(('v', index_ver))
    else:
        for index_hori in range(len(hori_pose_Go_stone_set)):
            instance_list.append(('h', index_hori))
        for index_ver in range(len(ver_pose_Go_stone_set)):
            instance_list.append(('v', index_ver))
    RGB_image_dict = dict()
    Depth_image_dict = dict()
    mask_dict = dict()
    image_array = np.array(cv2_img, dtype=np.float32)
    depth_array = np.array(cv2_depth_img, dtype=np.float32)
    for instance_index in instance_list:
        if instance_index[0]=='v':
            mask_array = np.load('/home/terry/Mask_RCNN/samples/stones/mask/ver_'+str(instance_index[1])+'.npy')
        else:
            mask_array = np.load('/home/terry/Mask_RCNN/samples/stones/mask/hori_'+str(instance_index[1])+'.npy')
        center_position = [int((np.min(np.where(mask_array==1)[0])+np.max(np.where(mask_array==1)[0]))/2.0), int((np.min(np.where(mask_array==1)[1])+np.max(np.where(mask_array==1)[1]))/2.0)]
        cut_range = [0,0,0,0]
        if center_position[0]-60>=0 and center_position[0]+60<480:
            cut_range[0], cut_range[1] = center_position[0]-60, center_position[0]+60
        elif center_position[0]-60<0:
            cut_range[0], cut_range[1] = 0, 120
        elif center_position[0]+60>=480:
            cut_range[0], cut_range[1] = 360, 480
        if center_position[1]-80>=0 and center_position[1]+80<640:
            cut_range[2], cut_range[3] = center_position[1]-80, center_position[1]+80
        elif center_position[1]-80<0:
            cut_range[2], cut_range[3] = 0, 160
        elif center_position[1]+80>=640:
            cut_range[2], cut_range[3] = 480, 640  
        input_mask_index_small = mask_array[cut_range[0]:cut_range[1], cut_range[2]:cut_range[3]][np.newaxis,:,:,np.newaxis].swapaxes(1,3).swapaxes(2,3)
        input_image_index_small = image_array[cut_range[0]:cut_range[1], cut_range[2]:cut_range[3],:][np.newaxis,:,:,:].swapaxes(1,3).swapaxes(2,3)
        input_depth_index_small = depth_array[cut_range[0]:cut_range[1], cut_range[2]:cut_range[3]][np.newaxis,:,:,np.newaxis].swapaxes(1,3).swapaxes(2,3)
        RGB_image_dict[instance_index]=input_image_index_small/256.0
        Depth_image_dict[instance_index]=input_depth_index_small/256.0
        mask_dict[instance_index]=input_mask_index_small.astype(np.float32)
    score_dict = dict()
    for instance_index in instance_list:
        for action_name in ['ad_hoc_pick', 'dig_grasp', 'scoop_dir0', 'scoop_dir1', 'scoop_dir2', 'scoop_dir3', 'scoop_dir4', 'scoop_dir5', 'scoop_dir6', 'scoop_dir7']:
            net_dict[action_name].eval()
            with torch.no_grad():
                score_dict[(instance_index, action_name)] = net_dict[action_name](torch.from_numpy(RGB_image_dict[instance_index]), torch.from_numpy(Depth_image_dict[instance_index]), torch.from_numpy(mask_dict[instance_index])).numpy()[0,0]
                score_dict[(instance_index, action_name)] = min(max(score_dict[(instance_index, action_name)], 0), 1)
                score_dict[(instance_index, action_name)] = round(score_dict[(instance_index, action_name)]*10.0)/10.0
    for k,v in score_dict.items():
	    if v<0.3:
	        del score_dict[k]
    print score_dict
    score_current_value_number=dict.fromkeys(np.round(np.arange(0.3, 1.1, 0.1), decimals=1).tolist(), 0)
    for k,v in score_dict.items():
        for i0 in np.round(np.arange(0.3, 1.1, 0.1), decimals=1):
            if v==i0:
                score_current_value_number[i0]+=1
    sorted_instance_action_pair = [i1[0] for i1 in sorted(score_dict.items(), lambda x, y: cmp (x[1], y[1]), reverse=True)]
    sorted_instance_action_pair_ppc = []
    current_i0_start = 0
    for i0 in np.round(np.arange(1.0, 0.2, -0.1), decimals=1):
        sorted_instance_action_pair_segment = sorted_instance_action_pair[current_i0_start:current_i0_start + score_current_value_number[i0]]
        random.shuffle(sorted_instance_action_pair_segment)
        sorted_instance_action_pair_ppc += sorted_instance_action_pair_segment
        current_i0_start+=score_current_value_number[i0]
    for i0 in np.arange(1.0, 0.2, -0.1):
        sorted_instance_action_pair_ppc += random.shuffle(sorted_instance_action_pair[current_i0_start:current_i0_start + score_current_value_number[i0]])
        current_i0_start+=score_current_value_number[i0]
    
    for instance_action_pair in sorted_instance_action_pair_ppc:
        if instance_action_pair[1]=='ad_hoc_pick':
            if instance_action_pair[0][0]=='h' and hori_pose_Go_stone_set[instance_action_pair[0][1]]==[None, None, None]:
                continue
            elif instance_action_pair[0][0]=='v' and ver_pose_Go_stone_set[instance_action_pair[0][1]]==[None, None, None]:
                continue
            if direct_pick_collision_check(instance_action_pair[0])==False:
                if (instance_action_pair[0][0]=='v' and ver_pose_Go_stone_set[instance_action_pair[0][1]][2]<0.075) or (instance_action_pair[0][0]=='h' and hori_pose_Go_stone_set[instance_action_pair[0][1]][2]<0.075):
                    direct_pick_execute(instance_action_pair[0])
                    whether_executed = True
                    executed_action_index = 0
                    go_to_end_place()
                    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                    Robotiq.goto(robotiq_client, pos=0.019, speed=0.5, force=10, block=True)
                    softtip()
                    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                    break
        elif instance_action_pair[1]=='dig_grasp':
            if instance_action_pair[0][0]=='v':
                continue
            if hori_pose_Go_stone_set[instance_action_pair[0][1]]==[None, None, None]:
                continue               
            if poke_grasp_collision_check(instance_action_pair[0])==False:
                if poke_grasp_execute(instance_action_pair[0])==True:
                    whether_executed = True
                    executed_action_index = 1
                    go_to_end_place()
                    rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                    Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                    softtip()
                    rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                    break 
        elif instance_action_pair[1].startswith('scoop_dir'):
            scooping_direction_index = int(instance_action_pair[1][-1])
            if scoop_collision_check(instance_action_pair[0], scooping_direction_index)==False:
                scoop_execute(instance_action_pair[0], scooping_direction_index) 
                whether_executed = True
                executed_action_index = scooping_direction_index+2
                go_to_end_place()
                rob.translate((0,0,-0.07), acc=0.3, vel=0.8, wait=True)
                Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)
                rospy.sleep(0.5)
                servo_pub.publish(180)
                Robotiq.goto(robotiq_client, pos=0.01, speed=0.5, force=10, block=True)
                rospy.sleep(1)
                servo_pub.publish(10)
                rospy.sleep(1)
                rob.translate((0,0,0.07), acc=0.3, vel=0.8, wait=True)
                break
    else:
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001
        if ver_pose_Go_stone_set == []:
            min_height = np.min(np.array(hori_pose_Go_stone_set)[:,2])
        elif hori_pose_Go_stone_set == []:
            min_height = np.min(np.array(ver_pose_Go_stone_set)[:,2])
        else:
            min_height = min(np.min(np.array(hori_pose_Go_stone_set)[:,2]), np.min(np.array(ver_pose_Go_stone_set)[:,2]))

        if min_height>0.058:
            hardtip()
            rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.04), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,0.04), acc=0.1, vel=0.4, wait=True) 
        elif min_height>0.053:
            hardtip()
            rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.05), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,0.05), acc=0.1, vel=0.4, wait=True) 
        elif min_height>0.045:
            hardtip()
            rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
            rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True) 
        else:
            if hori_pose_Go_stone_set==[]:
                hardtip()
                rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
                rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)      
            else:
                softtip()
                servo_pub.publish(100)
                if len(hori_pose_Go_stone_set)<=2:
                    sweep_index_set = range(len(hori_pose_Go_stone_set))
                else:
                    sweep_index_set = random.sample(range(len(hori_pose_Go_stone_set)), 2)
                for hori_index in sweep_index_set:
                    alpha = atan2(0.69-hori_pose_Go_stone_set[hori_index][1], 0.05-hori_pose_Go_stone_set[hori_index][0])
                    if alpha > pi/2 and alpha<=pi:
                        rotate_angle = alpha-3*pi/2
                    else:
                        rotate_angle = alpha+pi/2
                    print alpha, rotate_angle
                    rob.movel_tool((0,0,0,0,0,-rotate_angle), acc=0.1, vel=0.2, wait=True) 
                    world2tcp = rob.get_pose()
                    world2tcp.pos.x+=0.004
                    world2tcp.pos.y-=0.002
                    tcp2fingertip=m3d.Transform()
                    tcp2fingertip.pos.x = 0
                    tcp2fingertip.pos.y = 0  
                    tcp2fingertip.pos.z = 0.354
                    world2fingertip=(world2tcp*tcp2fingertip)
                    rob.translate((hori_pose_Go_stone_set[hori_index][0]-world2fingertip.pos.x,hori_pose_Go_stone_set[hori_index][1]-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
                    rob.movel_tool((0.002,0,0,0,0,0), acc=0.1, vel=0.2, wait=True) 
                    rob.set_tcp((0.002, 0, 0.354, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.movel_tool((0,0,0,0,20*pi/180,0), acc=0.3, vel=0.8, wait=True) 
                    rob.set_tcp((0, 0, 0, 0, 0, 0))
                    rospy.sleep(0.5)
                    rob.translate((0, 0, -0.05), acc=0.1, vel=0.3, wait=True)
                    rob.translate((0, 0, -0.05), acc=0.001, vel=0.02, wait=False)
                    wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                    ini_wrench = wrench.wrench.force.z
                    torque_memory=[]
                    while True:
                        wrench = rospy.wait_for_message('/robotiq_ft_wrench', WrenchStamped, timeout = None)
                        torque_memory.append(wrench.wrench.force.z)
                        if len(torque_memory)<9:
                            if wrench.wrench.force.z<ini_wrench-5:
                                rob.stopl()
                                break
                        else:
                            if wrench.wrench.force.z<max(torque_memory[0:9])-5:
                                rob.stopl()
                                break 
                    point_to_bowl_center_length=sqrt((hori_pose_Go_stone_set[hori_index][0]-0.05)**2+(hori_pose_Go_stone_set[hori_index][1]-0.69)**2)
                    rob.translate((point_to_bowl_center_length*0.8*cos(alpha), point_to_bowl_center_length*0.8*sin(alpha), 0), acc=0.1, vel=0.1, wait=True)
                    #hardtip()
                    rob.translate((0, 0, 0.1), acc=0.3, vel=0.8, wait=True)
                    rob.movej((radians(99.10),radians(-91.67), radians(-107.60), radians(-70.72), radians(89.13), radians(-80.76)), 0.3, 0.7)
    if whether_executed == True:
        while True:
            whether_succeed = raw_input("Whether_succeed? (Used for training.)")
            if whether_succeed == 'y':
                if os.path.exists('collecting_data/'+dir_name+'/success_probability_for_training.npy')==True:
                    success_probability_list = np.load('collecting_data/'+dir_name+'/success_probability_for_training.npy').tolist()
                    success_probability_list.append(1)
                    np.save('collecting_data/'+dir_name+'/success_probability_for_training.npy', np.array(success_probability_list))
                else:
                    np.save('collecting_data/'+dir_name+'/success_probability_for_training.npy', np.array([1]))
                break
            elif whether_succeed == 'n':
                while True:
                    which_failure = raw_input("Which_failure? (none objects-1, multiple objects-2)")
                    if which_failure == '2':
                        if os.path.exists('collecting_data/'+dir_name+'/success_probability_for_training.npy')==True:
                            success_probability_list = np.load('collecting_data/'+dir_name+'/success_probability_for_training.npy').tolist()
                            success_probability_list.append(0.2)
                            np.save('collecting_data/'+dir_name+'/success_probability_for_training.npy', np.array(success_probability_list))
                        else:
                            np.save('collecting_data/'+dir_name+'/success_probability_for_training.npy', np.array([0.2]))
                        break
                    elif which_failure == '1':
                        if os.path.exists('collecting_data/'+dir_name+'/success_probability_for_training.npy')==True:
                            success_probability_list = np.load('collecting_data/'+dir_name+'/success_probability_for_training.npy').tolist()
                            success_probability_list.append(0)
                            np.save('collecting_data/'+dir_name+'/success_probability_for_training.npy', np.array(success_probability_list))
                        else:
                            np.save('collecting_data/'+dir_name+'/success_probability_for_training.npy', np.array([0]))
                        break
                break
            else:
                continue
        while True:
            whether_succeed = raw_input("Whether_succeed? (Used for counting.)")
            if whether_succeed == 'y':
                if os.path.exists('collecting_data/'+dir_name+'/success_probability_for_counting.npy')==True:
                    success_probability_list = np.load('collecting_data/'+dir_name+'/success_probability_for_counting.npy').tolist()
                    success_probability_list.append(1)
                    np.save('collecting_data/'+dir_name+'/success_probability_for_counting.npy', np.array(success_probability_list))
                else:
                    np.save('collecting_data/'+dir_name+'/success_probability_for_counting.npy', np.array([1]))
                break
            elif whether_succeed == 'n':
                while True:
                    which_failure = raw_input("Which_failure? (none objects-1, multiple objects-2)")
                    if which_failure == '2':
                        if os.path.exists('collecting_data/'+dir_name+'/success_probability_for_counting.npy')==True:
                            success_probability_list = np.load('collecting_data/'+dir_name+'/success_probability_for_counting.npy').tolist()
                            success_probability_list.append(0.2)
                            np.save('collecting_data/'+dir_name+'/success_probability_for_counting.npy', np.array(success_probability_list))
                        else:
                            np.save('collecting_data/'+dir_name+'/success_probability_for_counting.npy', np.array([0.2]))
                        break
                    elif which_failure == '1':
                        if os.path.exists('collecting_data/'+dir_name+'/success_probability_for_counting.npy')==True:
                            success_probability_list = np.load('collecting_data/'+dir_name+'/success_probability_for_counting.npy').tolist()
                            success_probability_list.append(0)
                            np.save('collecting_data/'+dir_name+'/success_probability_for_counting.npy', np.array(success_probability_list))
                        else:
                            np.save('collecting_data/'+dir_name+'/success_probability_for_counting.npy', np.array([0]))
                        break
                break
            else:
                continue
        picking_count+=1
        cv2.imwrite('collecting_data/'+dir_name+'/image/'+str(picking_count)+'.jpeg', cv2_img)
        np.save('collecting_data/'+dir_name+'/image/'+str(picking_count)+'.npy', image_array)  
        cv2.imwrite('collecting_data/'+dir_name+'/depth/'+str(picking_count)+'.jpeg', cv2_depth_img)
        np.save('collecting_data/'+dir_name+'/depth/'+str(picking_count)+'.npy', depth_array)  
        np.save('collecting_data/'+dir_name+'/mask/'+str(picking_count)+'.npy', mask_array) 
        if os.path.exists('collecting_data/'+dir_name+'/executed_action_list.npy')==True:
            executed_action_list = np.load('collecting_data/'+dir_name+'/executed_action_list.npy').tolist()
            executed_action_list.append(executed_action_index)
            np.save('collecting_data/'+dir_name+'/executed_action_list.npy', np.array(executed_action_list))
        else:
            np.save('collecting_data/'+dir_name+'/executed_action_list.npy', np.array([executed_action_index]))  

                
                # which action131, 0.06473477762172797], [0.05002869920583822, 0.668622249134879, 0.06071871600676282], [0.06440163964727431, 
                # scooping direction

continue?1
started 0
dsag
0
4 [[0.04668653782347547, 0.6707378586377724, 0.06839232745120549], [0.03280455124331196, 0.7103714654178814, 0.06714785720277328], [0.06588207227622601, 0.7158708723604919, 0.061861484348545825], [0.05294131558489416, 0.6983723679689213, 0.06766596181565715]]
4 [[0.0474641837667751, 0.6389890776977984, 0.06108390836759686], [0.07231934859911011, 0.6799311189506201, 0.0653063875353524], [-0.0022380245619193512, 0.6886842237873709, 0.06411592964927121], [-0.0013676879704039007, 0.709979404700162, 0.06464461843809616]]
{(('v', 3), 'scoop_dir0'): 0, (('h', 0), 'scoop_dir2'): 0.30971497, (('v', 3), 'scoop_dir5'): 0, (('h', 1), 'ad_hoc_pick'): 0.0062750727, (('v', 2), 'ad_hoc_pick'): 1, (('v', 1), 'ad_hoc_pick'): 0.8805655, (('h', 1), 'scoop_dir2'): 0.580009, (('v', 1), 'scoop_dir3'): 0.032600865, (('v', 2), 'scoop_dir1'): 0, (('v', 0), 'ad_hoc_pick'): 0.11637674, (('h', 0), 'scoop_dir6'): 0.50948167, (('h', 3), 'scoop_dir4'): 0.5673703, (('v', 1), 'dig_grasp'): 0

KeyboardInterrupt: 

In [38]:
hard((0.075-0.07387900463012186)+(0.05-0.0491991223463534)+(0.025-0.023048922430698122)+(0.025-0.024825341678513236))/4.0

0.0010119022285783466

In [39]:
((0.675-0.6763793500101929)+(0.675-0.6752322836099283)+(0.7-0.6996321601604742)+(0.65-0.6493484668989968))/4.0

-0.00014806516989804286

In [20]:
Robotiq.goto(robotiq_client, pos=0.08, speed=0.000001, force=1, block=True)
#servo_pub.publish(180)
#softtip()

In [113]:
Robotiq.goto(robotiq_client, pos=0.8, speed=0.000001, force=1, block=True)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 291, in run
    data = self._get_data()
  File "/home/terry/anaconda3/envs/ros_control/lib/python2.7/site-packages/urx/ursecmon.py", line 338, in _get_data
    tmp = self._s_secondary.recv(1024)
timeout: timed out



In [81]:
131, 0.06473477762172797], [0.05002869920583822, 0.668622249134879, 0.06071871600676282], [0.06440163964727431, 
                    np.save('collecting_data/'+dir_
Robotiq.goto(robotiq_client, pos=0.023, speed=0.5, force=10, block=True)

rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,pi/5,0),acc=0.1, vel=0.2)
rob.movel_tool((0,0,0,0,-pi/5,0),acc=0.1, vel=0.2)

KeyboardInterrupt: 

In [25]:
go_to_safe_place()
softtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)

<Transform:
<Orientation: 
array([[-0.01305109,  0.99989524,  0.00625989],
       [ 0.99991361,  0.01304105,  0.00164169],
       [ 0.00155988,  0.00628078, -0.99997906]])>
<Vector: (0.04293, 0.85097, 0.45825)>
>

In [9]:
go_to_safe_place()
hardtip()
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
servo_pub.publish(10)    
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rospy.sleep(0.5)
rob.translate((0.05-pos_x,0.85-pos_y,0), acc=0.1, vel=0.4, wait=True)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
rob.translate_tool((0.0065,0,0),acc=0.1, vel=0.2)

<Transform:
<Orientation: 
array([[-0.01308933,  0.99989511,  0.00619996],
       [ 0.99991303,  0.01307907,  0.00169334],
       [ 0.00161207,  0.00622159, -0.99997935]])>
<Vector: (0.04282, 0.80750, 0.45825)>
>

In [23]:
go_to_safe_place()
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
rob.movel_tool((0,0,0,0,0,0*pi/180), acc=0.1, vel=0.2, wait=True)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
#rob.movel_tool((0,0,0,0,-20*pi/180,0), acc=0.1, vel=0.2, wait=True)
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)
hardtip()
#go_to_safe_place()
servo_pub.publish(0)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = 0.002
tcp2fingertip.pos.y = 0  
tcp2fingertip.pos.z = 0.356
world2tcp = rob.get_pose()
world2tcp.pos.x+=0.004
world2tcp.pos.y+=0.001
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.1, vel=0.4, wait=True)
servo_pub.publish(100)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)


<Transform:
<Orientation: 
array([[-0.01301363,  0.99989673,  0.00609763],
       [ 0.99991311,  0.01300053,  0.00218286],
       [ 0.00210337,  0.00612551, -0.99997903]])>
<Vector: (0.04382, 0.84645, 0.39055)>
>

In [21]:
rob.set_tcp((0.002, 0, 0.356, 0, 0, 0))
rospy.sleep(0.5)
#if np.cross([fingertip2normal[0], fingertip2normal[1]], [-sin(alpha), cos(alpha)])>0:
rob.movel_tool((0,0,0,0,-pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.movel_tool((0,0,0,0,pi/6,0), acc=0.3, vel=0.8, wait=True)
rob.set_tcp((0, 0, 0, 0, 0, 0))
rospy.sleep(0.5)

In [26]:
go_to_safe_place()
pos_x = rob.get_pose().pos.x+0.007
pos_y = rob.get_pose().pos.y-0.001
rob.translate((0.05-pos_x+0.02,0.69-pos_y-0.02,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,-0.06), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((-0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,-0.04,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0.04,0,0), acc=0.1, vel=0.4, wait=True)
rob.translate((0,0,0.06), acc=0.1, vel=0.4, wait=True)  

<Transform:
<Orientation: 
array([[-0.01328703,  0.99989138,  0.00637776],
       [ 0.99991118,  0.0132801 ,  0.00112777],
       [ 0.00104295,  0.00639218, -0.99997903]])>
<Vector: (0.06282, 0.67096, 0.45798)>
>

In [37]:
        hardtip()
        go_to_safe_place()
        pos_x = rob.get_pose().pos.x+0.007
        pos_y = rob.get_pose().pos.y-0.001

        rob.translate((0.05-pos_x+0.03,0.69-pos_y-0.03,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,-0.075), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((-0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,-0.06,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0.06,0,0), acc=0.1, vel=0.4, wait=True)
        rob.translate((0,0,0.075), acc=0.01, vel=0.05, wait=True)     


<Transform:
<Orientation: 
array([[-0.01329421,  0.99989335,  0.00604565],
       [ 0.99990707,  0.01327565,  0.00310065],
       [ 0.00302006,  0.00608631, -0.99997692]])>
<Vector: (0.07290, 0.66112, 0.45890)>
>

In [5]:
Robotiq.goto(robotiq_client, pos=0.023, speed=0.1, force=10, block=True)

In [ ]:
go_to_safe_place()
Robotiq.goto(robotiq_client, pos=5*0.14/48.0+1.12/48.0, speed=0.1, force=10, block=True)
servo_pub.publish(0)
rob.movel_tool((0,0,0,0,0,0),acc=0.1, vel=0.2)
tcp2fingertip=m3d.Transform()
tcp2fingertip.pos.x = -0.021 # need change
tcp2fingertip.pos.z = 0.342 # need change
world2tcp = rob.get_pose()
world2tcp.pos.x+=0
world2tcp.pos.y-=0
world2fingertip=(world2tcp*tcp2fingertip)
rob.translate((0.05-world2fingertip.pos.x,0.85-world2fingertip.pos.y,0), acc=0.3, vel=0.8, wait=True)
rob.translate((0,0,-0.068), acc=0.1, vel=0.4, wait=True)

In [5]:
    eeTcam = m3d.Transform()
    eeTcam.pos = (0.076173+0.05, -0.0934057-0.03, 0.0074811)
    eeTcam_e = tf.transformations.euler_from_quaternion([-0.0143125,0.69183,-0.0012,0.722039], axes='sxyz')
    eeTcam.orient.rotate_xb(eeTcam_e[0])
    eeTcam.orient.rotate_yb(eeTcam_e[1])
    eeTcam.orient.rotate_zb(eeTcam_e[2])
    tcpTee = np.array([[0,-1,0,0], [0,0,-1,0], [1,0,0,0], [0,0,0,1]])
    tcp2cam = np.matmul(tcpTee, eeTcam.get_matrix())
    print tcp2cam

[[ 0.02153275 -0.9995875  -0.01900464  0.1234057 ]
 [ 0.99884698  0.02232484 -0.04250071 -0.0074811 ]
 [ 0.04290745 -0.01806757  0.99891567  0.126173  ]
 [ 0.          0.          0.          1.        ]]


In [62]:
softtip()

In [22]:
print np.load('/home/terry/catkin_ws/src/scoop/src/collecting_data/Mon_Nov_23_19_36_07_2020/executed_action_list.npy')

[[0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 1 0 0]]


In [8]:
go_to_home()

In [27]:
import time
#device = 'cuda:0'
time0 = time.time()

input1 = torch.randn(1, 3, 60, 80)
input2 = torch.randn(1, 1, 60, 80)
input3 = torch.randn(1, 1, 60, 80)
print input1.dtype
with torch.no_grad():
    output = net_dict['ad_hoc_pick'](input1, input2, input3)
    print output.numpy()[0,0]
    print time.time()-time0

torch.float32
0.32871175
0.137641906738


In [24]:

mask_array=np.load('/home/terry/catkin_ws/src/scoop/src/collecting_data/Fri_Dec__4_14_21_46_2020/mask/1.npy')
image=np.load('/home/terry/catkin_ws/src/scoop/src/collecting_data/Fri_Dec__4_14_21_46_2020/depth/1.npy')
center_position = [int((np.min(np.where(mask_array==1)[0])+np.max(np.where(mask_array==1)[0]))/2.0), int((np.min(np.where(mask_array==1)[1])+np.max(np.where(mask_array==1)[1]))/2.0)]
cut_range = [0,0,0,0]
print center_position
if center_position[0]-60>=0 and center_position[0]+60<480:
    cut_range[0], cut_range[1] = center_position[0]-60, center_position[0]+60
elif center_position[0]-60<0:
    cut_range[0], cut_range[1] = 0, 120
elif center_position[0]+60>=480:
    cut_range[0], cut_range[1] = 360, 480
if center_position[1]-80>=0 and center_position[1]+80<640:
    cut_range[2], cut_range[3] = center_position[1]-80, center_position[1]+80
elif center_position[1]-80<0:
    cut_range[2], cut_range[3] = 0, 160
elif center_position[1]+80>=640:
    cut_range[2], cut_range[3] = 480, 640  
print cut_range
lll = image[cut_range[0]:cut_range[1], cut_range[2]:cut_range[3]]/256
print lll
cv2.imwrite('/home/terry/catkin_ws/src/scoop/src/collecting_data/Fri_Dec__4_14_21_46_2020/mask/aaa.jpg',lll)


[227, 329]
[167, 287, 249, 409]
[[0.66796875 0.66796875 0.66796875 ... 0.66796875 0.671875   0.671875  ]
 [0.66796875 0.66796875 0.66796875 ... 0.66796875 0.66796875 0.671875  ]
 [0.66796875 0.66796875 0.66796875 ... 0.66796875 0.66796875 0.671875  ]
 ...
 [0.6640625  0.6640625  0.6640625  ... 0.66796875 0.66796875 0.66796875]
 [0.6640625  0.6640625  0.6640625  ... 0.66796875 0.66796875 0.66796875]
 [0.6640625  0.6640625  0.6640625  ... 0.66796875 0.66796875 0.66796875]]


True

In [42]:
#input1 = torch.ones(1, 3, 100, 100)
#input2 = torch.ones(1, 1, 100, 100)
#input3 = torch.ones(1, 1, 100, 100)
test_data_x1 = np.load('/home/terry/catkin_ws/src/scoop/src/test_data_x1.npy')
test_data_x2 = np.load('/home/terry/catkin_ws/src/scoop/src/test_data_x2.npy')
test_data_x3 = np.load('/home/terry/catkin_ws/src/scoop/src/test_data_x3.npy')
print test_data_x1, test_data_x2, test_data_x3
input1 = torch.from_numpy(test_data_x1)
input2 = torch.from_numpy(test_data_x2)
input3 = torch.from_numpy(test_data_x3)
net_dict['ad_hoc_pick'].eval()
with torch.no_grad():
    output = net_dict['ad_hoc_pick'](input1, input2, input3)
output = torch.min(torch.max(output,torch.zeros(output.shape)),torch.ones(output.shape))
print(output.cpu())

[[[[0.46875    0.484375   0.4921875  ... 0.5625     0.5625
    0.5546875 ]
   [0.50390625 0.51171875 0.5234375  ... 0.5703125  0.5703125
    0.5703125 ]
   [0.5234375  0.53125    0.53515625 ... 0.5703125  0.5703125
    0.5703125 ]
   ...
   [0.51953125 0.50390625 0.4921875  ... 0.546875   0.546875
    0.54296875]
   [0.5390625  0.5234375  0.515625   ... 0.546875   0.546875
    0.54296875]
   [0.55078125 0.5390625  0.52734375 ... 0.546875   0.546875
    0.54296875]]

  [[0.484375   0.4921875  0.5        ... 0.5546875  0.5546875
    0.546875  ]
   [0.51171875 0.51953125 0.53125    ... 0.5546875  0.5546875
    0.5546875 ]
   [0.53125    0.5390625  0.54296875 ... 0.5546875  0.5546875
    0.5546875 ]
   ...
   [0.5234375  0.5078125  0.49609375 ... 0.53515625 0.53515625
    0.53125   ]
   [0.53125    0.515625   0.5078125  ... 0.53515625 0.53515625
    0.53125   ]
   [0.54296875 0.53125    0.51953125 ... 0.53515625 0.53515625
    0.53125   ]]

  [[0.5859375  0.59375    0.6015625  ... 0.640625

tensor([[0.0125],
        [0.0030],
        [0.0041],
        [0.0037],
        [0.0087],
        [0.0051],
        [0.0084],
        [0.0068],
        [0.0023],
        [0.0054],
        [0.0070],
        [0.0000],
        [0.0062],
        [0.0066],
        [0.0040],
        [0.0146],
        [0.0042],
        [0.0014],
        [0.0096],
        [0.0022],
        [0.0079],
        [0.0035],
        [0.0044],
        [0.0045],
        [0.0080],
        [0.0068],
        [0.0095],
        [0.0062],
        [0.0033],
        [0.0053],
        [0.0053],
        [0.0136],
        [0.0075]])


In [23]:
softtip()

In [7]:
import numpy as np
aaa=np.load('/home/terry/catkin_ws/src/scoop/src/collecting_data_20201204_afternoon/Fri_Dec__4_16_47_51_2020/success_probability_for_counting.npy')
print aaa.shape
print np.argwhere(aaa==0).shape


(85,)
(9, 1)
